# Import Necessatry Modules

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import openpyxl

# Loading WebDriver

In [2]:
ser = Service('/home/irfan/Downloads/chromedriver_linux64')

# Inititalizing Driver

In [3]:
driver = Chrome(service=ser)
driver.maximize_window()

## Reditrecting to https://www.alluniversity.info/pakistan/ WebSite

In [4]:
driver.get('https://www.alluniversity.info/pakistan/')

## Scrape the Name + URL of All Website leading top 10 Universities

In [5]:
top10 = {}
universities = {}


def loop_over_data(li, d, index=0):
    for link in li:
        anchor = link.find_element(By.TAG_NAME, 'a')
        anc = anchor.get_attribute('href')
        name = link.text[index:]
        if anc not in top10:
            d[name] = anc


def scrape_date():
    try:
        for i in range(3):
            ul = driver.find_elements(By.TAG_NAME, 'ul')
            if i == 0:
                top = ul[0].find_elements(By.TAG_NAME, 'li')
                loop_over_data(top, top10, 3)
                uni = ul[i+1].find_elements(By.TAG_NAME, 'li')
            else:
                uni = ul[0].find_elements(By.TAG_NAME, 'li')
            loop_over_data(uni, universities)
            next_page_link = driver.find_element(By.XPATH, "//a[@title='Next Page']")
            next_page_link.click()
    except NoSuchElementException:
        return

scrape_date()

## Scrape Required Information of Indvidual University

In [6]:
top = {'Ranking': [], 'Year Established': [], 'Address': [], 'Phone': [], 'Fax': [], 'Website': []}
other = {'Year Established': [], 'Address': [], 'Phone': [], 'Fax': [], 'Website': []}

def loop_over_uni(lst, dict):
    for link in lst:
        driver.get(link)
        tables = driver.find_elements(By.TAG_NAME, 'table')
        for table in tables:
            tr = table.find_elements(By.TAG_NAME, 'tr')
            for row in tr:
                th = row.find_element(By.TAG_NAME, 'th').text
                try:
                    anchor = row.find_element(By.TAG_NAME, 'a')
                    dict[th].append(anchor.get_attribute('href'))
                except:
                    td = row.find_element(By.TAG_NAME, 'td')
                    if len(dict) == 10:
                        span = td.find_element(By.TAG_NAME, 'span')
                        dict[th].append(span.text)
                    else:
                        if th in dict:
                            dict[th].append(td.text)
        if len(dict['Fax']) != len(dict['Address']):
            dict['Fax'].append('')
        if len(dict['Website']) != len(dict['Address']):
            dict['Website'].append('')
        if len(dict['Phone']) != len(dict['Address']):
            dict['Phone'].append('')
loop_over_uni(list(top10.values()), top)
loop_over_uni(list(universities.values()), other)

# Saving Data into Excel File

In [16]:
workbook = openpyxl.Workbook()
worksheet = workbook.active
def AddDataIntoExcel(dict, file, uni_name):
    headers = list(dict.keys())
    r = 1
    worksheet.cell(row=r, column=1).value = 'University'
    for i in range(len(headers)):
        worksheet.cell(row=r, column=i+2).value = headers[i]
    r += 1
    names = list(uni_name.keys())
    for name in names:
        worksheet.cell(row=r, column=1).value = name
        r += 1
    r -= len(names)
    c = 2
    for key, value in dict.items():
        for data in value:
            worksheet.cell(row=r, column=c).value = data
            r += 1
        c += 1
        r -= len(value)
    workbook.save(f'{file}.xlsx')

AddDataIntoExcel(top, 'Top10', top10)
AddDataIntoExcel(other, 'OtherUni', universities)

# Loading Data From Excel File

In [23]:
df = pd.read_excel('Top10.xlsx')
frame = df.fillna('-')
frame

,University,Ranking,Year Established,Address,Phone,Fax,Website
0,"National University of Sciences & Technology, ...",#1,1991,"Sector H-12, Islamabad, Islamabad Capital Terr...",+92 (51) 9085 1101,+92 (51) 9085 1102,https://www.nust.edu.pk/Pages/Default.aspx
1,"Quaid-i-Azam University, Islamabad",#2,1965,"PO Box 1090, Islamabad, Islamabad Capital Terr...",+92 (51) 921 9877,+92 (51) 921 9888,https://qau.edu.pk/
2,Pakistan Institute of Engineering and Applied ...,#3,1997,"Lehtrar Road, Nilore, Islamabad, Pakistan","+92 (51) 1111 74327, +92 (51) 924 8611-20",-,http://www.pieas.edu.pk/
3,"Lahore University of Management Sciences, Lahore",#4,1985,"Sector U, D.H.A, Lahore, Punjab, Pakistan",+92 (42) 572 2670 9,+92 (42) 572 2591,https://lums.edu.pk/
4,"University of Engineering & Technology, Lahore...",#5,1923,"Lahore Grand Trunk Road, Lahore, Punjab, Pakistan",+92 (42) 682 2012,+92 (42) 682 2566,https://www.uet.edu.pk/
5,"University of Peshawar, Pakistan, Peshawar",#6,1950,"Jamrud Road, Peshawar, North West Frontier Pro...",+92 (91) 921 6469,+92 (91) 921 6470,-
6,"University of the Punjab, Lahore",#7,1882,"New Campus, Lahore, Punjab, Pakistan",+92 (42) 923 1102,+92 (42) 923 1103,http://www.pu.edu.pk/
7,COMSATS Institute of Information Technology Is...,#8,1998,"Park Road, Tarlai Kalan, Islamabad 45550, Paki...",-,-,https://www.comsats.edu.pk/
8,"University of Lahore, Lahore",#9,1999,"1 Km Raiwind Road, Lahore, Punjab, Pakistan",+92 423 7515460-7,+92 423 7515519,http://www.uol.edu.pk/
9,"University of Agriculture, Faisalabad, Faisal...",#10,1909,"Ch. Muhammad Hussain, Faisalabad, Punjab, Paki...",+92 (41) 920 0161,+92 (41) 920 0764,http://www.uaf.edu.pk/


In [22]:
dff = pd.read_excel('OtherUni.xlsx')
droped_frame = dff.drop('Website.1', axis=1)
final_frame = droped_frame.fillna('-')
final_frame

,University,Year Established,Address,Phone,Fax,Website
0,"Abasyn University, Peshawar",2007.0,"Ring road, Peshawar, North West Frontier Provi...",+92 (91) 2250820,+92 (91) 2582834,http://www.abasyn.edu.pk/
1,Abbottabad University of Science and Technolog...,2015.0,"Havelian, Khyber Pakhtunkhwa, Pakistan, Abbott...",92 992 402117,92 (992) 402 117,https://aust.edu.pk/
2,"Abdul Wali Khan University, Mardan",2009.0,"Nowshera Mardan Rd, Mardan, পাকিস্তান, Mardan,...",92 937 9230640,92-937-843381,-
3,"Aga Khan University, Karachi",1983.0,"Stadium Road, PO Box 3500, Karachi, Sindh, Pak...",+92 (21) 493 0051,+92 (21) 493 4294,https://www.aku.edu/Pages/home.aspx
4,"Air University, Islamabad",2002.0,"E-9, PAF Complex, Islamabad, Islamabad Capital...",+92 (51) 920 5801,+92 (51) 920 0158,http://www.au.edu.pk/
...,...,...,...,...,...,...
139,"Virtual University of Pakistan, Lahore",-,"M.A. JINNAH CAMPUS, DEFENCE ROAD, OFF RAIWIND ...",92 42 111 880 880,"92-42-99200604, 99202174",https://www.vu.edu.pk/
140,"Women University Mardan, Mardan",-,"Mardan, Khyber Pakhtunkhwa, Pakistan, Mardan, ...",92 937 9230640,-,https://awkum.edu.pk/
141,"Women University Multan, Multan,",-,"Chowk Kuchery, Multan, পাকিস্তান, Multan, , Pa...",-,-,https://www.wum.edu.pk/
142,"Women University of Azad Jammu and Kashmir, Ba...",-,"Kotli, Azad Jammu & KashmirAzad Kashmir, Pakis...",05-823-960034,05-823-960034,http://www.wuajk.edu.pk/
